# Vanilla Binary RNN Model

This notebook is used to train a simple Vanilla RNN model for Binary classification of Piano and Electric Guitar. We chose the following two classes because there numbers are really close to each other, hence would be able to solve the class imbalance issue beforehand and dont have to worry about it.

### Importing Required Libraries

This section will import the required libaries that will be used to actually implement the training for the Vanilla RNN Binary Classification Model

In [1]:
import numpy as np
import pandas as pd
import os
import glob
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Input
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import time

C:\Users\Saksham Goel\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Adding this so that can use the python scripts for loading the training data

In [2]:
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

Importing the python script training data loader function. This function loads the data from the *.wav* files directly

In [3]:
from py_scripts.directory_funcs import *
from py_scripts.wav_file_funcs import *
from py_scripts.misc_audio_signal_funcs import *
from py_scripts.raw_training_data_creation import load_irmas_data

### Loading Training Data

This section uses the script from the training data creation module to load the training data.

In [4]:
# First define the training data directory, and find the different classes that exist
training_data_dir = '../../data/whole_dataset/training/'
get_subdirectory_names(training_data_dir)

['cel', 'cla', 'flu', 'gac', 'gel', 'org', 'pia', 'sax', 'tru', 'vio', 'voi']

In [5]:
# Define the classes that we will work
all_classes = get_subdirectory_names(training_data_dir)
# Defining the classes that will be the ones on which we will train for the project
classes_for_project = ['gel', 'pia', 'sax', 'vio', 'voi']
mapping_to_index = dict(zip(classes_for_project, range(len(classes_for_project))))
# Getting the frequency for each class
class_num_files = dict(zip(classes_for_project, [len(get_file_names(construct_path(training_data_dir, class_name), '*.wav')) for class_name in classes_for_project]))
# Defining the class which will be used as the one v/s all classifier to denote all other classes except the current class

In [6]:
# Defining the various parameters for loading the input data
rnn_window = (300, 300) # in the format of length of vector and the shift
class_num_examples = 300
X, y = load_irmas_data(training_data_dir, classes_for_project, rnn_window, number_of_training_examples_per_class=class_num_examples)

Getting Data from gel
Processing: 300 files
Loaded all the data from the class
Getting Data from pia
Processing: 300 files
Loaded all the data from the class
Getting Data from sax
Processing: 300 files
Loaded all the data from the class
Getting Data from vio
Processing: 300 files
Loaded all the data from the class
Getting Data from voi
Processing: 300 files
Loaded all the data from the class


In [7]:
# Verifying the shape
X.shape, y.shape

((6000, 294, 300), (6000,))

In [8]:
# Doing a train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
# Freeing up previous memory
X = None
y = None

In [10]:
print(f'Total Number of Training Samples: {y_train.shape}')
print(f'Total number of timestamp values for each sample: {X_train.shape[1]}')
print(f'Total number of features for each sample: {X_train.shape[-1]}')
#print(f'Minimum Feature Value: {np.min(X_train)}, Maximum Feature Value: {np.max(X_train)}')

Total Number of Training Samples: (4800,)
Total number of timestamp values for each sample: 294
Total number of features for each sample: 300


In [11]:
print(f'Total Number of Testing Samples: {y_test.shape}')
print(f'Total number of timestamp values for each sample: {X_test.shape[1]}')
print(f'Total number of features for each sample: {X_test.shape[-1]}')
#print(f'Minimum Feature Value: {np.min(X_train)}, Maximum Feature Value: {np.max(X_train)}')

Total Number of Testing Samples: (1200,)
Total number of timestamp values for each sample: 294
Total number of features for each sample: 300


In [12]:
y_train_categorical = pd.Categorical(y_train)
y_train_numerical = y_train_categorical.codes
y_test_categorical = pd.Categorical(y_test)
y_test_numerical = y_test_categorical.codes
# Checking the categries
print(y_train_categorical.categories, y_test_categorical.categories)

Index(['gel', 'pia', 'sax', 'vio', 'voi'], dtype='object') Index(['gel', 'pia', 'sax', 'vio', 'voi'], dtype='object')


In [13]:
# Find the number of categories
len(y_train_categorical.categories)

5

In [14]:
# If training for more than 1 class then need to convert to categorical
if len(y_train_categorical.categories) > 2:
    y_train_numerical = to_categorical(y_train_numerical)
    y_test_numerical = to_categorical(y_test_numerical)

In [15]:
y_train_numerical.shape

(4800, 5)

## Check For Memory Usage

In [16]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('X_train', 1693440128),
 ('X_test', 423360128),
 ('y_train', 57696),
 ('y_test', 14496),
 ('y_train_categorical', 5284),
 ('y_test_categorical', 1684),
 ('Dense', 1056),
 ('EarlyStopping', 1056),
 ('Embedding', 1056),
 ('ModelCheckpoint', 1056),
 ('Sequential', 1056),
 ('SimpleRNN', 1056),
 ('TensorBoard', 1056),
 ('nb_dir', 285),
 ('class_num_files', 240),
 ('mapping_to_index', 240),
 ('all_classes', 160),
 ('training_data_dir', 153),
 ('Input', 136),
 ('check_output_directory', 136),
 ('construct_path', 136),
 ('exist_directory', 136),
 ('exist_file', 136),
 ('get_directory_contents', 136),
 ('get_file_names', 136),
 ('get_left_channel_data', 136),
 ('get_right_channel_data', 136),
 ('get_sound_signals', 136),
 ('get_subdirectory_names', 136),
 ('load_irmas_data', 136),
 ('normalize_sound_signals', 136),
 ('read_wav_file', 136),
 ('shift_sound_signals', 136),
 ('to_categorical', 136),
 ('train_test_split', 136),
 ('y_test_numerical', 112),
 ('y_train_numerical', 112),
 ('classes_fo

## Model Definition

This section will define the model architecture that will be used for the training purposes

In [17]:
# Defining the parameters for the Embedding layer
number_of_features = X_train.shape[-1]
number_of_time_stamps = X_train.shape[1]
print(f'Number of Features (Feature Vector Length): {number_of_features}, Number of Time Stamps: {number_of_time_stamps}')

Number of Features (Feature Vector Length): 300, Number of Time Stamps: 294


In [37]:
# Define the model
rnn_layer_num_units = 50
num_classes_for_training = len(y_train_categorical.categories)
model = Sequential()
model.add(SimpleRNN(rnn_layer_num_units, input_shape=(number_of_time_stamps, number_of_features), dropout = 0.1))
model.add(Dense(num_classes_for_training, activation='softmax'))
# Compiling the model
model.compile(loss= 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_8 (SimpleRNN)     (None, 50)                17550     
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 255       
Total params: 17,805
Trainable params: 17,805
Non-trainable params: 0
_________________________________________________________________


In [38]:
# Adding a checkpoint
parent_weight_save_dir = '../../data/Training Results/Vanilla Simple RNN/Weights'
tensor_board_dir_path = '../../data/Training Results/Vanilla Simple RNN/TensorBoard'
check_output_directory(parent_weight_save_dir)
current_experiment_name = f'AllClasses_InputVectorLen-{number_of_features}_TimeStamps-{number_of_time_stamps}_CT-{time.time()}'
weight_file_path = os.path.join(parent_weight_save_dir, f'{current_experiment_name}.hdf5')
tensor_board_file_path = os.path.join(tensor_board_dir_path, current_experiment_name)
check_output_directory(tensor_board_file_path)
checkpoint = ModelCheckpoint(weight_file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tensorboard = TensorBoard(log_dir=tensor_board_file_path)
early_stopping_criteria = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=0, mode='auto')
callbacks_list = [tensorboard, checkpoint, early_stopping_criteria]

In [39]:
history = model.fit(X_train, y_train_numerical, epochs=50, batch_size=64, validation_split=0.2, callbacks=callbacks_list)

Train on 3840 samples, validate on 960 samples
Epoch 1/50
3840/3840 [==============================] - 14s 4ms/step - loss: 1.5908 - acc: 0.2602 - val_loss: 1.5387 - val_acc: 0.2979

Epoch 00001: val_acc improved from -inf to 0.29792, saving model to ../../data/Training Results/Vanilla Simple RNN/Weights\AllClasses_InputVectorLen-300_TimeStamps-294_CT-1544152149.7949927.hdf5
Epoch 2/50
3840/3840 [==============================] - 17s 4ms/step - loss: 1.5378 - acc: 0.3081 - val_loss: 1.5635 - val_acc: 0.3177

Epoch 00002: val_acc improved from 0.29792 to 0.31771, saving model to ../../data/Training Results/Vanilla Simple RNN/Weights\AllClasses_InputVectorLen-300_TimeStamps-294_CT-1544152149.7949927.hdf5
Epoch 3/50
3840/3840 [==============================] - 16s 4ms/step - loss: 1.4372 - acc: 0.3576 - val_loss: 1.5245 - val_acc: 0.3010

Epoch 00003: val_acc did not improve from 0.31771
Epoch 4/50
3840/3840 [==============================] - 17s 4ms/step - loss: 1.4226 - acc: 0.3753 - va

In [40]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test_numerical, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

1200/1200 [==============================] - 3s 3ms/step
Accuracy: 36.25%


# RNN Model Initialization with different layers

In [68]:
# Define the model
rnn_layer_num_units = 75
num_classes_for_training = len(y_train_categorical.categories)
model = Sequential()
model.add(SimpleRNN(rnn_layer_num_units, input_shape=(number_of_time_stamps, number_of_features), dropout = 0.1))
model.add(Dense(30, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(num_classes_for_training, activation='softmax'))
# Compiling the model
model.compile(loss= 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_17 (SimpleRNN)    (None, 75)                28200     
_________________________________________________________________
dense_23 (Dense)             (None, 30)                2280      
_________________________________________________________________
dense_24 (Dense)             (None, 20)                620       
_________________________________________________________________
dense_25 (Dense)             (None, 5)                 105       
Total params: 31,205
Trainable params: 31,205
Non-trainable params: 0
_________________________________________________________________


In [69]:
# Adding a checkpoint
parent_weight_save_dir = '../../data/Training Results/Vanilla Simple RNN/Weights'
tensor_board_dir_path = '../../data/Training Results/Vanilla Simple RNN/TensorBoard'
check_output_directory(parent_weight_save_dir)
current_experiment_name = f'AllClasses_HiddenUnits-{rnn_layer_num_units}_InputVectorLen-{number_of_features}_TimeStamps-{number_of_time_stamps}_CT-{time.time()}'
weight_file_path = os.path.join(parent_weight_save_dir, f'{current_experiment_name}.hdf5')
tensor_board_file_path = os.path.join(tensor_board_dir_path, current_experiment_name)
check_output_directory(tensor_board_file_path)
checkpoint = ModelCheckpoint(weight_file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tensorboard = TensorBoard(log_dir=tensor_board_file_path)
early_stopping_criteria = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=0, mode='auto')
callbacks_list = [tensorboard, checkpoint, early_stopping_criteria]

In [70]:
history = model.fit(X_train, y_train_numerical, epochs=50, batch_size=64, validation_split=0.2, callbacks=callbacks_list)

Train on 3840 samples, validate on 960 samples
Epoch 1/50
3840/3840 [==============================] - 23s 6ms/step - loss: 1.5529 - acc: 0.2750 - val_loss: 1.5423 - val_acc: 0.2573

Epoch 00001: val_acc improved from -inf to 0.25729, saving model to ../../data/Training Results/Vanilla Simple RNN/Weights\AllClasses_HiddenUnits-75_InputVectorLen-300_TimeStamps-294_CT-1544159872.6018162.hdf5
Epoch 2/50
3840/3840 [==============================] - 22s 6ms/step - loss: 1.3936 - acc: 0.3544 - val_loss: 1.4241 - val_acc: 0.2938

Epoch 00002: val_acc improved from 0.25729 to 0.29375, saving model to ../../data/Training Results/Vanilla Simple RNN/Weights\AllClasses_HiddenUnits-75_InputVectorLen-300_TimeStamps-294_CT-1544159872.6018162.hdf5
Epoch 3/50
3840/3840 [==============================] - 22s 6ms/step - loss: 1.2778 - acc: 0.4008 - val_loss: 1.3953 - val_acc: 0.3365

Epoch 00003: val_acc improved from 0.29375 to 0.33646, saving model to ../../data/Training Results/Vanilla Simple RNN/Weig

In [71]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test_numerical, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

1200/1200 [==============================] - 6s 5ms/step
Accuracy: 37.67%
